<div class='alert alert-info' style='text-align: center'><h1>Manual VOI LUT (window) an MR Image</h1>
- yet another MR processing notebook -</div>

#### The idea is to adjust the contrast of MR images prior to conversion to 8 bit to better separate tissues of like-density.
- This mimics what radiologists do with the WW/WL tool in a DICOM image viewer.
- It helps to distinguish tumors from other brain tissue that surrounds them.
- Since tumors are typically more dense than the stuff around them, but only slightly, we can make them more 'enhanced'.
- The effect is more pronounced since the skull bones have been MIP'd out of the images.
- Must be done before the pixels are crunched down to 8 bit.

1. **Determine the pixel range of the image.**

2. **Create a VOI LUT from raw pixels.**

3. **Apply the LUT to the image.**

4. **Rejoice!**

### - Do imports and define some functions

In [ ]:
import os
import numpy as np
import pandas as pd
import pydicom
import matplotlib.pyplot as plt

In [ ]:
# Make a simple linear VOI LUT from the raw (stored) pixel data
def make_lut(pixels, width, center, p_i):
    
    # Slope and Intercept set to 1 and 0 for MR. Get these from DICOM tags instead if using 
    # on a modality that requires them (CT, PT etc)
    slope = 1.0
    intercept = 0.0
    min_pixel = int(np.amin(pixels))
    max_pixel = int(np.amax(pixels))

    # Make an empty array for the LUT the size of the pixel 'width' in the raw pixel data
    lut = [0] * (max_pixel + 1)
    
    # Invert pixels and cent for MONOCHROME1. We invert the specified center so that 
    # increasing the center value makes the images brighter regardless of photometric intrepretation
    invert = False
    if p_i == "MONOCHROME1":
        invert = True
    else:
        center = (max_pixel - min_pixel) - center
        
    # Loop through the pixels and calculate each LUT value
    for pix_value in range(min_pixel, max_pixel):
        lut_value = pix_value * slope + intercept
        voi_value = (((lut_value - center) /  width + 0.5) * 255.0)
        clamped_value = min(max(voi_value, 0), 255)
        if invert:
            lut[pix_value] = round(255 - clamped_value)
        else:
            lut[pix_value] = round(clamped_value)
        
    return lut

In [ ]:
# Apply the LUT to a pixel array
def apply_lut(pixels_in, lut):
    
    pixels_in = pixels_in.flatten()
    pixels_out = [0] * len(pixels_in)
    
    for i in range(0, len(pixels_in)):
        pixel = pixels_in[i]
        pixels_out[i] = int(lut[pixel])
        
    return pixels_out

### - Load and display an image

In [ ]:
from pydicom.pixel_data_handlers.util import apply_voi_lut
# Load an image
image = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00148/T1wCE/Image-95.dcm')
pixels = image.pixel_array

# Print out the pixel 'width'
print("Min pixel value: " + str(np.min(pixels)))
print("Max pixel value: " + str(np.max(pixels)))

print(image.PhotometricInterpretation)


plt.figure(figsize= (6,6))
plt.imshow(pixels, cmap='gray');

In [ ]:
# Plot a histogram of the raw pixel data
fig, axes = plt.subplots(nrows=1, ncols=1,sharex=False, sharey=False, figsize=(10,4))
plt.title('Pixel Range: ' + str(np.min(pixels)) + '-' + str(np.max(pixels)))
plt.hist(pixels.ravel(), np.max(pixels), (1, np.max(pixels)))
plt.tight_layout()
plt.show()

- The pixel range or width, is from 0 to 2857. That's 2858 shades of gray.
- Since we can't display such a 'wide' image on our consumer grade display adapters and monitors, we need to bin the pixels to a usable range (8 bit).

### - Make a LUT
- If we specify **window_center = window_width / 2** as the center, the image will appear as the default image above does.
- Lowering the width has the effect of *decreasing the contrast* by binning pixels.
- Lowering the center has the effect of *decreasing the brightness*.

### - Now we'll tweak the window values to produce a more contrasty image.

In [ ]:
# Apply three different WW/WL settings via LUT. We'll set the center slightly less than half to adjust for brightness.
window_width_1 = np.max(image.pixel_array)
window_center_1 = window_width_1 / 2

lut = make_lut(image.pixel_array, window_width_1, window_center_1, image.PhotometricInterpretation)
image1 = np.reshape(apply_lut(pixels, lut), (pixels.shape[0],pixels.shape[1]))

window_width_2 = 450
window_center_2 = 450

lut = make_lut(image.pixel_array, window_width_2, window_center_2, image.PhotometricInterpretation)
image2 = np.reshape(apply_lut(pixels, lut), (pixels.shape[0],pixels.shape[1]))

window_width_3 = 900
window_center_3 = 90

lut = make_lut(image.pixel_array, window_width_3, window_center_3, image.PhotometricInterpretation)
image3 = np.reshape(apply_lut(pixels, lut), (pixels.shape[0],pixels.shape[1]))

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2,sharex=True, sharey=True, figsize=(12, 12))
ax = axes.ravel()
ax[0].set_title('Default Image')
ax[0].imshow(image.pixel_array, cmap='gray')
ax[1].set_title(f'Width: {window_width_1} / Center: {window_center_1}')
ax[1].imshow(image1, cmap='gray')
ax[2].set_title(f'Width: {window_width_2} / Center: {window_center_2}')
ax[2].imshow(image2, cmap='gray')
ax[3].set_title(f'Width: {window_width_3} / Center: {window_center_3}')
ax[3].imshow(image3, cmap='gray')
plt.tight_layout()
plt.show()

### - Result: 
#### - The tumor area is more contrasted against the surrounding brain tissue. Especially in the bottom left image.
#### - There are fine details visible in this image that are obscured on the other images.

### - Let's load another image and try the same process.
- Just copy the last 3 cells and change the filename.

In [ ]:
# Load an image
image = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00014/FLAIR/Image-126.dcm')
pixels = image.pixel_array

# Print out the pixel 'width'
print("Min pixel value: " + str(np.min(pixels)))
print("Max pixel value: " + str(np.max(pixels)))

plt.figure(figsize= (6,6))
plt.imshow(pixels, cmap='gray');

### - This image doesn't have as wide of pixel range as the first image does. 
- We'll set it's level a little bit lower and try to make the tumor stand out better.

In [ ]:
# Apply three different WW/WL settings via LUT. We'll set the level slightly less than half to adjust for brightness.
window_width_1 = 1000
window_center_1 = 900

lut = make_lut(image.pixel_array, window_width_1, window_center_1, image.PhotometricInterpretation)
image1 = np.reshape(apply_lut(pixels, lut), (pixels.shape[0],pixels.shape[1]))

window_width_2 = 600
window_width_2 = 900

lut = make_lut(image.pixel_array, window_width_2, window_center_2, image.PhotometricInterpretation)
image2 = np.reshape(apply_lut(pixels, lut), (pixels.shape[0],pixels.shape[1]))

window_width_3 = 300
window_center_3 = 900

lut = make_lut(image.pixel_array, window_width_3, window_center_3, image.PhotometricInterpretation)
image3 = np.reshape(apply_lut(pixels, lut), (pixels.shape[0],pixels.shape[1]))

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2,sharex=True, sharey=True, figsize=(12, 12))
ax = axes.ravel()
ax[0].set_title('Default Image')
ax[0].imshow(image.pixel_array, cmap='gray')
ax[1].set_title(f'Width: {window_width_1} / Center: {window_center_1}')
ax[1].imshow(image1, cmap='gray')
ax[2].set_title(f'Width: {window_width_2} / Center: {window_center_2}')
ax[2].imshow(image2, cmap='gray')
ax[3].set_title(f'Width: {window_width_3} / Center: {window_center_3}')
ax[3].imshow(image3, cmap='gray')
plt.tight_layout()
plt.show()

### Again, we can clearly see the tumor better. I suspect CNNs will too!

#### Some of my other MR notebooks
- Tumor Object Detection -> https://www.kaggle.com/davidbroberts/brain-tumor-object-detection
- Determining MR image planes -> https://www.kaggle.com/davidbroberts/determining-mr-image-planes
- Determining MR Slice Orientation -> https://www.kaggle.com/davidbroberts/determining-mr-slice-orientation
- Determining DICOM image order -> https://www.kaggle.com/davidbroberts/determining-dicom-image-order
- Reference Lines on MR images -> https://www.kaggle.com/davidbroberts/mr-reference-lines
- Standardizing MR Images -> https://www.kaggle.com/davidbroberts/standardizing-mr-images
- Export DICOM Images by Plane -> https://www.kaggle.com/davidbroberts/export-dicom-series-by-plane